# Hubble Ultra Deep Field Displays

## Importing Libraries

In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

rcParams['figure.figsize'] = [10. , 8.]

## Importing Ultra Deep Field Image Data

In [ ]:
data = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
data = data.byteswap(inplace=True).newbyteorder()

## Plotting Image

In [ ]:
m , s = np.mean(data) , np.std(data) #setting the mean and standard deviation of sources
plt.imshow(data, interpolation='nearest', cmap='gray' , vmin=m-s, vmax=m+s, origin='lower') #displaying the image
plt.colorbar() #generating a reference colorbar
plt.savefig('stars') #saving figure

## Finding Background Noise to Take Out of Signal Data

In [ ]:
bkg = sep.Background(data)

In [ ]:
print(bkg.globalback) #displaying background data's
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

## Plotting the Background

In [ ]:
plt.imshow(bkg_image , interpolation='nearest', cmap='gray', origin='lower') #Displaying the background
plt.colorbar()
plt.savefig('bkg_image')

## Plotting the Background Noise

In [ ]:
bkg_rms = bkg.rms() #creating a variable for noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin = 'lower') #Displaying noise
plt.colorbar()
plt.savefig('bkg_noise')

## Obtaining Image Data with Background Reduced

In [ ]:
data_sub = data - bkg #subtracting out background from image data

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) #obtaining sources of objects in the image without noise

In [ ]:
print('Individual Sources: ', len(objects))

## Plotting Source Image

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots() #creating a sublplot
m , s = np.mean(data_sub) , np.std(data_sub) #getting mean and standard deviation values for reduced data
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower') #plotting the image

for i in range(len(objects)): #plotting ellipses around each source
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i], angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none') #keeping center of ellipse clear so we can see the sources
    e.set_edgecolor('red') #setting ellipse color
    ax.add_artist(e)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)
#Getting the flux, error in flux, and a flag variable for major outlier values

In [ ]:
plt.hist(flux, bins=100) #plotting flux histogram
plt.yscale('log') #setting to a log scale
plt.title('Flux Graph') #creating the title

In [ ]:
hdu = fits.open('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
hdu.info() #getting the header of the Ultra Deep Field Image